In [1]:
import numpy as np
import pandas as pd

In [42]:
df = pd.read_csv('Books.csv', header=None, names=['ISBN', 'Book-Title', 'Book-Author', 'Year-Of-Publication', 'Publisher', 'Image-URL-S', 'Image-URL-M', 'Image-URL-L'])

C:\Users\dell\AppData\Local\Temp\ipykernel_4244\941292330.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('Books.csv', header=None, names=['ISBN', 'Book-Title', 'Book-Author', 'Year-Of-Publication', 'Publisher', 'Image-URL-S', 'Image-URL-M', 'Image-URL-L'])


In [43]:
df.sample()

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
270582,0746003080,Journey to the Lost Temple (Puzzle Adventures),S. Leigh,1989,E.D.C. Publishing,http://images.amazon.com/images/P/0746003080.0...,http://images.amazon.com/images/P/0746003080.0...,http://images.amazon.com/images/P/0746003080.0...


In [48]:
df.rename(columns={'Book-Title':'title','Book-Author':'author','Year-Of-Publication':'year','Publisher':'publisher'},inplace=True)

In [51]:
df = df[['ISBN','title','author','year','publisher']]

In [53]:
df.head(2)

,ISBN,title,author,year,publisher
0,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher
1,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press


In [54]:
user = pd.read_csv('Users.csv', header=None, names=['User-ID','Location','Age'])

C:\Users\dell\AppData\Local\Temp\ipykernel_4244\873421714.py:1: DtypeWarning: Columns (0,2) have mixed types. Specify dtype option on import or set low_memory=False.
  user = pd.read_csv('Users.csv', header=None, names=['User-ID','Location','Age'])


In [56]:
user.head()

,User-ID,Location,Age
0,User-ID,Location,Age
1,1,"nyc, new york, usa",NaN
2,2,"stockton, california, usa",18.0
3,3,"moscow, yukon territory, russia",NaN
4,4,"porto, v.n.gaia, portugal",17.0


In [57]:
user.rename(columns={'User-ID':'user-id','Location':'location','Age':'age'},inplace=True)

In [58]:
user.head(3)

,user-id,location,age
0,User-ID,Location,Age
1,1,"nyc, new york, usa",NaN
2,2,"stockton, california, usa",18.0


In [90]:
ratings = pd.read_csv('Ratings.csv', header=None, names=['User-ID','ISBN','Book-Rating'])

C:\Users\dell\AppData\Local\Temp\ipykernel_4244\1404735639.py:1: DtypeWarning: Columns (0,2) have mixed types. Specify dtype option on import or set low_memory=False.
  ratings = pd.read_csv('Ratings.csv', header=None, names=['User-ID','ISBN','Book-Rating'])


In [91]:
ratings.rename(columns={'User-ID':'user-id','Book-Rating':'rating'},inplace=True)

In [92]:
ratings

,user-id,ISBN,rating
0,User-ID,ISBN,Book-Rating
1,276725,034545104X,0
2,276726,0155061224,5
3,276727,0446520802,0
4,276729,052165615X,3
...,...,...,...
1149776,276704,1563526298,9
1149777,276706,0679447156,0
1149778,276709,0515107662,10
1149779,276721,0590442449,10


In [93]:
df.shape

(271361, 5)

In [94]:
user.shape

(278859, 3)

In [95]:
ratings.shape

(1149781, 3)

In [96]:
x= ratings['user-id'].value_counts()>200

In [97]:
y= x[x].index

In [98]:
y

Index([ '11676',   198711,   153662,    98391,  '35859',   212898, '278418',
          76352,   110973,   235105,
       ...
         260183,   '9856',   155916,    73681,   268622,  '44296',   274808,
        '28634',  '59727',   188951],
      dtype='object', name='user-id', length=899)

In [99]:
ratings= ratings[ratings['user-id'].isin(y)]

In [100]:
ratings.head(2)

,user-id,ISBN,rating
1457,277427,002542730X,10
1458,277427,0026217457,0


In [103]:
ratings_with_books= ratings.merge(df,on='ISBN')

In [104]:
ratings_with_books.shape

(487498, 7)

In [110]:
no_rating= ratings_with_books.groupby('title')['rating'].count().reset_index().rename(columns={'rating':'no of rating'})

In [113]:
final_rating= ratings_with_books.merge(no_rating,on='title')

In [115]:
final_rating.head(3)

,user-id,ISBN,rating,title,author,year,publisher,no of rating
0,277427,002542730X,10,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,82
1,3363,002542730X,0,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,82
2,11676,002542730X,6,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,82


In [118]:
final_rating= final_rating[final_rating['no of rating']>=50]

In [121]:
final_rating.drop_duplicates(['user-id','title'],inplace=True)

C:\Users\dell\AppData\Local\Temp\ipykernel_4244\3886136945.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_rating.drop_duplicates(['user-id','title'],inplace=True)


In [123]:
final_rating.shape

(59838, 8)

In [129]:
book_pivot= final_rating.pivot_table(columns='user-id',index='title',values='rating',aggfunc='count')

In [131]:
book_pivot.fillna(0,inplace=True)

In [133]:
book_pivot.shape

(742, 888)

In [134]:
from scipy.sparse import csr_matrix
book_sparse = csr_matrix(book_pivot)

In [148]:
type(book_sparse)

scipy.sparse._csr.csr_matrix

In [147]:
from sklearn.neighbors import NearestNeighbors

# Assuming 'book_sparse' is your sparse matrix
model = NearestNeighbors(algorithm='brute')  # 'auto' selects the most efficient algorithm based on the data

# Fit the model to your sparse matrix
model.fit(book_sparse)

NearestNeighbors(algorithm='brute')

In [151]:
distances, suggestiobns = model.kneighbors(book_pivot.iloc[237,:].values.reshape(1,-1), n_neighbors=6)

In [152]:
distances

array([[ 0.        ,  9.89949494, 10.09950494, 11.91637529, 12.04159458,
        12.12435565]])

In [153]:
suggestiobns 

array([[237, 240, 238, 239, 242, 241]], dtype=int64)

In [155]:
for i in range(len(suggestiobns)):
    print(book_pivot.index[suggestiobns [i]])

Index(['Harry Potter and the Chamber of Secrets (Book 2)',
       'Harry Potter and the Prisoner of Azkaban (Book 3)',
       'Harry Potter and the Goblet of Fire (Book 4)',
       'Harry Potter and the Order of the Phoenix (Book 5)',
       'Harry Potter and the Sorcerer's Stone (Harry Potter (Paperback))',
       'Harry Potter and the Sorcerer's Stone (Book 1)'],
      dtype='object', name='title')


In [156]:
np.where(book_pivot.index=='Animal Farm')[0][0]

54

In [161]:
def recommend_book(book_name):
    book_id = np.where(book_pivot.index== book_name)[0][0]
    distances, suggestiobns = model.kneighbors(book_pivot.iloc[book_id,:].values.reshape(1,-1), n_neighbors=6)


    for i in range(len(suggestiobns)):
        print(book_pivot.index[suggestiobns [i]])

In [162]:
recommend_book('Animal Farm')

Index(['Animal Farm',
       'Slaughterhouse Five or the Children's Crusade: A Duty Dance With Death',
       'Jacob Have I Loved', 'The Dead Zone',
       'Like Water for Chocolate: A Novel in Monthly Installments, With Recipes, Romances, and Home Remedies',
       'A Day Late and a Dollar Short'],
      dtype='object', name='title')
